In [1]:
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import geopandas as gpd
import pandas as pd
import glob
import zipfile
import rasterio
import os
import xarray as xr
import json
from rasterstats import zonal_stats
from rasterio.transform import from_origin


Populating the interactive namespace from numpy and matplotlib


In [2]:
print(os.getcwd())
from pathlib import Path
# folder = Path(r'../Data') # assumes working directory is onhm-fetcher-parser
folder = Path(r'../Data_v1_1') # assumes working directory is onhm-fetcher-parser
print(folder)
# shapefiles = folder.glob("*_0[1-2].shp")
shapefiles = folder.glob("*.shp")
gdf = pd.concat([
    gpd.read_file(shp)
    for shp in shapefiles
]).pipe(gpd.GeoDataFrame)
gdf.reset_index(drop=True, inplace=True)
# gdf.plot()
print(gdf)

B:\GitRepos\onhm-fetcher-parser\notebooks
..\Data_v1_1
            GM_LAYER         LAYER            GM_TYPE  OBJECTID  hru_segmen  \
0       NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type         1        1551   
1       NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type         2        1551   
2       NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type         3        1547   
3       NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type         3        1547   
4       NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type         4        1555   
...              ...           ...                ...       ...         ...   
140767  NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type    114951         294   
140768  NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type    114952         285   
140769  NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type    114953         226   
140770  NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type    114954         294   
140771  NHM\nhru_v11  NHM\nhru_v11  Unknown Area Type    114955         115 

In [3]:
import requests
from requests.exceptions import HTTPError
from datetime import datetime, timedelta
from urllib.parse import urlencode

prcpurl = 'https://thredds.daac.ornl.gov/thredds/ncss/daymet-v3-agg/na.ncml'
prcppayload = {
#     'var': 'lat&var=lon&var=tmax',
    'var': 'lat&var=lon&var=prcp&var=srad&var=swe&var=tmax&var=tmin&var=vp',
    'north': '54',
    'west': '-126',
    'east': '-65',
    'south': '23',
    'disableProjSubset': 'on',
    'horizStride': '1',
    'time_start': '2018-12-31T12:00:00Z',
    'time_end': '2018-12-31T12:00:00Z',
    'timeStride': '1',
    'accept': 'netcdf'}    
try:
    s = requests.Session()
    #https://github.com/psf/requests/issues/1454
    qry = urlencode(prcppayload).replace('%26','&')
    qry = qry.replace('%3D', '=')
    print(qry)
    tmaxfile = requests.get(prcpurl, params=qry)
    tmaxfile.raise_for_status()
except HTTPError as http_err:
    print(f'HTTP error occured: {http_err}')
except Exception as err:
    print(f'Other error occured: {err}')
else:
    print('Gridmet data retrieved!')

var=lat&var=lon&var=prcp&var=srad&var=swe&var=tmax&var=tmin&var=vp&north=54&west=-126&east=-65&south=23&disableProjSubset=on&horizStride=1&time_start=2018-12-31T12%3A00%3A00Z&time_end=2018-12-31T12%3A00%3A00Z&timeStride=1&accept=netcdf
Gridmet data retrieved!


In [4]:
with open('tmax_test2.nc', 'wb') as fh:
    fh.write(tmaxfile.content)
fh.close


<function BufferedWriter.close>

In [5]:
#=========================================================
#            MACAV2METDATA FILE PARAMETERS
#=========================================================
# dirPath='https://thredds.daac.ornl.gov/thredds/ncss/ornldaac/1328/2018/daymet_v3_prcp_2018_na.nc4?var=lat&var=lon&var=prcp&north=52.880049298000074+&west=-124.72462483099997+&east=-66.94953853699997+&south=24.839424370000074&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2018-12-31T00:00:00Z&time_end=2018-12-31T00:00:00Z&timeStride=1&accept=netcdf'
# fileName='/thredds/dodsC/MET/tmmx/tmmx_2019.nc'
dirPath = 'tmax_test2.nc'
# dirPath = 'daymet_v3_tmax_2018_na.nc4.nc'
#--------------------------------------------------------
#   FORM FILENAME AND GET HANDLE TO FILE AND DATA
#--------------------------------------------------------
fullfilename= dirPath
print(fullfilename)

ds = xr.open_dataset(fullfilename)

print(ds)

# df = ds.to_dataframe()

print('\n The meta data is: \n', ds.attrs)
lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['time']
datahandle=ds['tmax']
dhlat = ds['lat']
dhlon = ds['lon']
crshandle=ds['lambert_conformal_conic']
print('\n The crs meta data is \n', crshandle.attrs)
print(datahandle)
# crstransform = crshandle.attrs['GeoTransform']
# print(crstransform)

#collect data to describe geotransform
lonmin = float(ds.attrs['geospatial_lon_min'])
latmax = float(ds.attrs['geospatial_lat_max'])
# lonres = float(ds.attrs['geospatial_lon_resolution'])
# latres = float(ds.attrs['geospatial_lon_resolution'])

#Print some information on the data

print('\n Data attributes, sizes, and coords \n') 
print('\n Data attributes are: \n',datahandle.attrs)
print('\n Data sizes are: \n', datahandle.sizes)
print('\n Data coords are: \n', datahandle.coords)
print('\n Lat coords are: \n', dhlat.attrs)

ts = datahandle.sizes
print(type(ts))
print(ts['time'])
dayshape = ts['time']
Lonshape = ts['x']
Latshape = ts['y']
#dayshape,lonshape,latshape = datahandle.values.shape
print(dayshape, Lonshape, Latshape)

# datahandle.values[dayshape-1,:,:].shape

# print(lathandle.values.shape)
# print(type(lathandle.values))
# print(datahandle.dtype)
# print(np.isfortran(datahandle.values))

tmax_test2.nc
<xarray.Dataset>
Dimensions:                  (time: 1, x: 5904, y: 3377)
Coordinates:
  * y                        (y) float32 1687.0 1686.0 ... -1688.0 -1689.0
  * x                        (x) float32 -2650.25 -2649.25 ... 3251.75 3252.75
  * time                     (time) datetime64[ns] 2018-12-31
Data variables:
    lat                      (y, x) float32 ...
    lambert_conformal_conic  int16 ...
    lon                      (y, x) float32 ...
    prcp                     (time, y, x) float32 ...
    srad                     (time, y, x) float32 ...
    swe                      (time, y, x) float32 ...
    tmax                     (time, y, x) float32 ...
    tmin                     (time, y, x) float32 ...
    vp                       (time, y, x) float32 ...
Attributes:
    _NCProperties:       version=1|netcdflibversion=4.4.1|hdf5libversion=1.8.17
    start_year:          1980
    source:              Daymet Software Version 3.0
    Version_software:    Daymet S

In [7]:
# add tmax column to dataframe
gdf['tmax']=0.0

#open weight data
#wght_df = pd.read_csv('../Data/hru_metdata_weights.csv')
#wght_df_40 = pd.read_csv('../Data/hru_metdata_weights_40m.csv')
#wght_df_500 = pd.read_csv('../Data/hru_metdata_weights_500m.csv')
wght_UofI = pd.read_csv('../pkg/tmp_weights2.csv')
# print(len(wght_df['hru_id_nat'].unique()), len(wght_df_40['hru_id_nat'].unique()), 
#       len(wght_df_500['hru_id_nat'].unique()), len(wght_UofI['hru_id_nat'].unique()))
print(wght_UofI.head())

#iterate through hru's, grab all weights associated with hru_id, get total weighted value from netcdf file, assign to tmax
ndata = datahandle.values[dayshape-1,:,:].flatten(order='K')
# ndata=np.nan_to_num(data)
print(ndata[1000:])
# def w_mean(data)
unique_hru_ids = wght_UofI.groupby('nhm_id')
print(len(gdf), len(unique_hru_ids))

def get_wval(grp, ndata):
    ttmax = twght = 0.0
    for index, row in grp.iterrows():
        ttmax += row['w']*ndata[np.int(row['grid_ids'])]
        twght += row['w']
    return ttmax/twght
def np_get_wval(grp, ndata):
    return np.average(ndata[grp['grid_ids'].values.astype(int)], weights=grp['w'])
def np_get_wval2(grp, ndata):
    mdata = np.ma.masked_array(ndata[grp['grid_ids'].values.astype(int)], np.isnan(ndata[grp['grid_ids'].values.astype(int)]))
    return np.ma.average(mdata, weights=grp['w'])
#     return np.average(ndata[grp['grid_ids'].values.astype(int)], weights=grp['w'])
    
# unique_hru_ids.get_group(gdf['hru_id_nat']).agg({'tmax': np_get_wval(weight_id_rows, ndata)})    
td = np.zeros(len(gdf.index))
for index, row in gdf.iterrows():
    weight_id_rows = unique_hru_ids.get_group(row['nhm_id'])
#     print(weight_id_rows['grid_ids'].values.astype(int))
#     unique_hru_ids.get_group(row['hru_id_nat']).agg({'tmax': np_get_wval(weight_id_rows, ndata)})
#     gdf.loc[gdf.index[index],'tmax'] = np_get_wval(weight_id_rows, ndata)-273.5
    tmp = np_get_wval2(weight_id_rows, ndata)
    if index == 1:
        print(type(tmp))
    td[index] = np_get_wval2(weight_id_rows, ndata)
#     if td[index] < 0.0:
#         print(ndata[weight_id_rows['grid_ids'].values.astype(int)], weight_id_rows['w'])
#     print(index, td[index])
#     if row['hru_id_nat'] == 829:
#         print("in test")
#         for i2, el in weight_id_rows.iterrows():
#             print(el['w'], ndata[el['grid_ids'].astype(int)])
#         print(np.average(ndata[weight_id_rows['grid_ids'].values.astype(int)], weights=weight_id_rows['w'])-273.5)
#     print(index, row['hru_id_nat'], np_get_wval(weight_id_rows, ndata)-273.5)
#     gdf.loc[gdf.index[index], 'tmax'] =
# #     print(get_wval(weight_id_rows, ndata)-273.5)
# #     row.loc['tmax']=get_wval(weight_id_rows, ndata)-273.5
# #     gdf.loc[gdf.index[index], 'tmax'] = get_wval(weight_id_rows, ndata)-273.5
# print(len(td))
# gdf['tmax'] = gpd.GeoSeries([np.transpose(td)], index=gdf.index)
gdf['tmax'] = td.tolist()
gdf['tmax'].fillna(0.0)
# print(td.tolist())
print('min/max', gdf['tmax'].min(), gdf['tmax'].max())
# print(gdf)
# gdf.plot(figsize=(12,12), column = 'tmax',linewidth=0.25, edgecolor='white')    
# print(gdf.groupby(tmax).min)
f, ax = plt.subplots(2, figsize=(12,12))
gdf.plot(ax=ax[0], column = 'tmax',linewidth=0., edgecolor='white', scheme='quantiles')
ptmax = ds.air_temperature-273.5
ptmax_1 = ptmax.isel(day=dayshape-1)
lvs = np.arange(gdf['tmax'].min(), gdf['tmax'].max(), 0.5)
ptmax_1.plot(ax=ax[1], levels=lvs, cmap='viridis')

   grid_ids  nhm_id  hru_id_nat         w
0    585567   74427       74427  0.022644
1    585568   74427       74427  0.729424
2    585566   74427       74427  0.247932
3    585567   74450       74450  0.011813
4    585565   74450       74450  0.077199
[-6.5 -6.5 -6.5 ...  nan  nan  nan]
140772 113310
<class 'numpy.float64'>


C:\Users\rmcd\anaconda3\envs\ofp_env_upd\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Warning: converting a masked element to nan.


KeyError: 113594